<a href="https://colab.research.google.com/github/tanmayee2002/Music-Generation-via-ML/blob/main/MusicGeneration(Rock).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import re #(re - regular expression) python library primarily used for string searching and manipulation 
import numpy as np 
import pandas as pd
import music21 #python music library
import glob #to find files of the same extension within a directory
from IPython.display import Audio
import pickle #converts data into byte streams
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten


In [ ]:
!apt install fluidsynth

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 34 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ub

In [ ]:
from music21 import converter, instrument, note, chord, stream

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
songs = glob.glob('/content/drive/MyDrive/Data/Jazz/*mid')

print(songs)

['/content/drive/MyDrive/Data/Jazz/Jazz_AndreandSchwandt-aLittleDream.mid', '/content/drive/MyDrive/Data/Jazz/Jazz_Berckman-Berckman.mid', '/content/drive/MyDrive/Data/Jazz/Jazz_CameronLeeSimpson-.mid']


In [ ]:
def get_notes():
  notes = []
  for song in songs:
    midi = converter.parse(song) # midi format into a single stream object
    to_parse = []
    try:
      parts = instrument.partitionByInstrument(midi) #divides sounds based on diff instruments and joins those of the same instrument in a single stream
    except:
      pass
    if parts:
      to_parse = parts.parts[0].recurse() #gets into the Score's elements
    else:
      to_parse = midi.flat.notes #gets a stream of all notes in Score
    for element in to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch)) #seperates the pitch from the chord and appends it to the notes file
      elif (isinstance(element, chord.Chord)): 
        notes.append('.'.join(str(n) for n in element.normalOrder))

    with open("/content/drive/MyDrive/Data/notes_", 'wb') as filepath:
        pickle.dump(notes, filepath)
    print(len(set(notes)))
    pitchnames = sorted(set(item for item in notes)) #extract unique pitches
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    print(note_to_int)
    return notes


In [ ]:
print(get_notes())

15
{'A4': 0, 'B-4': 1, 'B4': 2, 'C4': 3, 'C5': 4, 'D4': 5, 'D5': 6, 'E-4': 7, 'E-5': 8, 'E4': 9, 'E5': 10, 'F#4': 11, 'F4': 12, 'G#4': 13, 'G4': 14}
['G4', 'F#4', 'G4', 'F#4', 'E4', 'G4', 'F#4', 'G4', 'F#4', 'G4', 'F#4', 'E4', 'G#4', 'B4', 'G#4', 'F4', 'E4', 'A4', 'G4', 'A4', 'G4', 'E4', 'A4', 'G4', 'E-4', 'G4', 'D4', 'E4', 'G4', 'A4', 'B4', 'E5', 'D5', 'G4', 'F#4', 'G4', 'F#4', 'E4', 'G4', 'F#4', 'G4', 'F#4', 'G4', 'F#4', 'E4', 'G#4', 'B4', 'G#4', 'F4', 'E4', 'A4', 'G4', 'A4', 'G4', 'E4', 'A4', 'G4', 'E-4', 'G4', 'D4', 'E4', 'G4', 'A4', 'B-4', 'A4', 'G4', 'G4', 'A4', 'B4', 'D5', 'E-5', 'C5', 'E-5', 'C5', 'B-4', 'G4', 'B-4', 'C4', 'D4', 'E-5', 'C5', 'E-5', 'C5', 'B-4', 'E-5', 'C5', 'E-5', 'C5', 'B-4', 'G4', 'B-4', 'C4', 'D4', 'E-5', 'B-4', 'C5', 'D5', 'G4', 'F#4', 'G4', 'F#4', 'E4', 'G4', 'F#4', 'G4', 'F#4', 'G4', 'F#4', 'E4', 'G#4', 'B4', 'G#4', 'F4', 'E4', 'A4', 'G4', 'A4', 'G4', 'E4', 'A4', 'G4', 'E-4', 'G4', 'D4', 'E4', 'G4', 'A4', 'B-4', 'A4', 'E4', 'G4', 'A4', 'B4', 'C5', 'B4', '

In [ ]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100
    pitchnames = sorted(set(item for item in notes)) #extract unique pitches
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) #creats a dictionary to map pitches and integers
    network_input = []
    network_output = []
    for i in range(0, len(notes) - sequence_length,  1):
        sequence_in = notes[i: i + sequence_length] #creates input sequence
        sequence_out = notes[i + sequence_length] #creates output sequence
        network_input.append([note_to_int[char] for char in sequence_in]) 
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)
    # reshape input and normalize input to make it compatible for the model
    print(network_output)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))    
    network_input = network_input / float(n_vocab)    
     
    network_output = np_utils.to_categorical(network_output) #converts object integers to binary class matrix
    
    return (network_input, network_output)


In [ ]:

def create_network(network_in, n_vocab): 
    model = Sequential()    #groups a linear stack of layers into an object with training and inference features
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))   
    model.add(Dropout(0.2))      #dropout layer randomly sets input units to 0 with a frequency of rate at each step, which helps to prevent overfitting
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())     #converts the shape of the layer into a single column
    model.add(Dense(256))   #dense layer is a fully connected layer
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))   #predicts a multinomial probability distribution
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


In [ ]:
from keras.callbacks import ModelCheckpoint
def train(model, network_input, network_output, epochs): 
   
    # Create checkpoint to save the best model weights.
    filepath = '/content/drive/MyDrive/Data/bestweightsJazz.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose=0, save_best_only=True)
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])

In [ ]:
def train_network():
    
    
    epochs = 150
    
    notes = get_notes()
    
    print('Notes processed')
    
    n_vocab = len(set(notes))
    print('Vocab generated')
    
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    model = create_network(network_in, n_vocab)
    print('Model created')
    print('Training in progress')
    train(model, network_in, network_out, epochs)
    print('Training completed')
    return model


In [ ]:
if __name__ == "__main__":
    train_network()

15
{'A4': 0, 'B-4': 1, 'B4': 2, 'C4': 3, 'C5': 4, 'D4': 5, 'D5': 6, 'E-4': 7, 'E-5': 8, 'E4': 9, 'E5': 10, 'F#4': 11, 'F4': 12, 'G#4': 13, 'G4': 14}
Notes processed
Vocab generated
[11, 9, 14, 11, 14, 11, 14, 11, 9, 13, 2, 13, 12, 9, 0, 14, 0, 14, 9, 0, 14, 7, 14, 5, 9, 14, 0, 1, 0, 9, 14, 0, 2, 4, 2, 6, 1, 2, 0, 1, 0, 14]
Input and Output processed
Model created
Training in progress
Epoch 1/150
2/2 [==============================] - 4s 235ms/step - loss: 2.5958
Epoch 2/150
2/2 [==============================] - 0s 229ms/step - loss: 3.2696
Epoch 3/150
2/2 [==============================] - 1s 217ms/step - loss: 2.3530
Epoch 4/150
2/2 [==============================] - 0s 220ms/step - loss: 2.1895
Epoch 5/150
2/2 [==============================] - 0s 217ms/step - loss: 2.1704
Epoch 6/150
2/2 [==============================] - 0s 200ms/step - loss: 2.1777
Epoch 7/150
2/2 [==============================] - 0s 218ms/step - loss: 2.0982
Epoch 8/150
2/2 [==============================] - 1s

In [ ]:

def get_inputSequences(notes, pitchnames, n_vocab):
    
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length] #charcters(notes) version of our input
        network_input.append([note_to_int[char] for char in sequence_in]) #from the dicitionary, appending the key of the character into network_input
    
    network_input = np.reshape(network_input, (len(network_input), 100, 1))
    
    return (network_input)



In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    # Pick a random integer
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # pick a random sequence from the input as a starting point for the prediction
    pattern = list(network_input[start])
    prediction_output = []
    print('Generating notes........')
    # generate 10(500 notes)
    print(len(pattern))
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    pat = np.asarray(pattern)
    for note_index in range(10):
        
        
        prediction_input = prediction_input / float(n_vocab)
        
        prediction = model.predict(prediction_input, verbose=0)
        
        
        index = np.argmax(prediction)
        # Mapping the predicted interger back to the corresponding note
        #result = int_to_note[index]
        # Storing the predicted output
        
        #prediction_output.append(result)
        
        index_list = np.array(index)
        
        
        np.append(pat, index)
        # Next input to the model
        pat = pat[1:len(pat)]
    for element in pat:
      prediction_output.append(int_to_note[element[0]])
    
    print('Notes Generated...')
    #print(pat)
    #print(prediction_output)
    return prediction_output

In [ ]:
def generate():
    
    #load the notes used to train the model
    with open('/content/drive/MyDrive/Data/notes_', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    
    # Get all pitch names
    n_vocab = len(set(notes))
    print(n_vocab)
    print('Initiating music generation process.......')
   
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    normalized_input = network_input / float(n_vocab)
    model = create_network(normalized_input, n_vocab)
    
    print('Loading Model weights.....')
    model.load_weights('/content/drive/MyDrive/Data/bestweights4.hdf5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    return prediction_output


In [ ]:
generate()

15
Initiating music generation process.......
Loading Model weights.....
Model Loaded
Generating notes........
100
Notes Generated...


['A4',
 'G4',
 'E4',
 'A4',
 'G4',
 'E-4',
 'G4',
 'D4',
 'E4',
 'G4',
 'A4',
 'B4',
 'E5',
 'D5',
 'G4',
 'F#4',
 'G4',
 'F#4',
 'E4',
 'G4',
 'F#4',
 'G4',
 'F#4',
 'G4',
 'F#4',
 'E4',
 'G#4',
 'B4',
 'G#4',
 'F4',
 'E4',
 'A4',
 'G4',
 'A4',
 'G4',
 'E4',
 'A4',
 'G4',
 'E-4',
 'G4',
 'D4',
 'E4',
 'G4',
 'A4',
 'B-4',
 'A4',
 'G4',
 'G4',
 'A4',
 'B4',
 'D5',
 'E-5',
 'C5',
 'E-5',
 'C5',
 'B-4',
 'G4',
 'B-4',
 'C4',
 'D4',
 'E-5',
 'C5',
 'E-5',
 'C5',
 'B-4',
 'E-5',
 'C5',
 'E-5',
 'C5',
 'B-4',
 'G4',
 'B-4',
 'C4',
 'D4',
 'E-5',
 'B-4',
 'C5',
 'D5',
 'G4',
 'F#4',
 'G4',
 'F#4',
 'E4',
 'G4',
 'F#4',
 'G4',
 'F#4',
 'G4',
 'F#4',
 'E4']

In [ ]:
def create_Pianomidi(prediction_output):  
  offset = 0 #time of the piece
  output_notes = []
  #create note and chord objects based on the values generated by the model
  #chords are a couple of notes played together(symbolized using '-' in our data) ! notes are a key and its pitch together
  for pattern in prediction_output:
    #print(output_notes)
    #separating chords and notes 
    if ('-' in pattern):   
      notes_in_chord = pattern.split('-')
      notes = []
      for current_note in notes_in_chord:
        if current_note.isdigit():
          new_note = note.Note(int(current_note))
        else:
          new_note = note.Note(current_note)        
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
        # pattern is a note
    else:          
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset += 0.5 # increase offset each iteration so that notes do not stack at one timestamp
  
  midi_stream = stream.Stream(output_notes) 
  print('Saving Output file as midi....')

  midi_stream.write('midi', fp='test_outputRock.mid')
    
prediction_op = generate()
print(create_Pianomidi(prediction_op))

15
Initiating music generation process.......
Loading Model weights.....
Model Loaded
Generating notes........
100
Notes Generated...
Saving Output file as midi....
None


In [ ]:
!find / -name *.sf2

find: ‘/proc/561/task/561/net’: Invalid argument
find: ‘/proc/561/net’: Invalid argument
find: ‘/proc/602/task/602/net’: Invalid argument
find: ‘/proc/602/net’: Invalid argument
/content/font.sf2


In [ ]:
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [ ]:
!fluidsynth -ni font.sf2 test_outputRock.mid -F outputRock.wav -r 44100

FluidSynth version 1.1.9
Copyright (C) 2000-2018 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Parameter 'test_outputRock.mid' not a SoundFont or MIDI file or error occurred identifying it.
No midi file specified!


In [ ]:
Audio('outputRock.wav')